In [1]:
import matplotlib.pyplot as plt
import numpy
import snap
import pickle
import copy
import sys
from sklearn.neural_network import MLPClassifier

In [2]:
city_network = snap.LoadEdgeList(snap.PUNGraph, "city_network_by_traffic.txt", 0, 1)
raw_gtd_data = pickle.load(open('raw_gtd_data.pkl', 'rb'))
gtd_group_dict = pickle.load(open('gtd_group_dict.pkl', 'rb'))
city_node_dict = pickle.load(open('city_node_dict.pkl', 'rb'))
city_list = pickle.load(open('city_list.pkl', 'rb'))

In [3]:
city_num_terror_connected = [{} for _ in range(1971, 2016)]
city_num_non_terror_connected = [{} for _ in range(1971, 2016)]
city_betweenness_centrality = [{} for _ in range(1971, 2016)]
city_degree_centrality = [{} for _ in range(1971, 2016)]
city_closeness_centrality = [{} for _ in range(1971, 2016)]
city_farness_centrality = [{} for _ in range(1971, 2016)]
city_connected = [{} for _ in range(1971, 2016)]
city_terror_occur = [{} for _ in range(1971, 2016)]
city_terror_recover = [{} for _ in range(1971, 2016)]

In [4]:
gtd_event_city_by_year = {}
for key, years in gtd_group_dict.iteritems():
	for year, months in years.iteritems():
		for month, events in months.iteritems():
			for event in events:
				if int(year) not in gtd_event_city_by_year:
					gtd_event_city_by_year[int(year)] = set()
				try:
					gtd_event_city_by_year[int(year)].add(city_node_dict[raw_gtd_data[event]['city']])
				except:
					pass

In [5]:
Nodes = snap.TIntFltH()
Edges = snap.TIntPrFltH()
snap.GetBetweennessCentr(city_network, Nodes, Edges, 1.0)

In [6]:
city_node_list = []
for key, value in city_node_dict.iteritems():
	city_node_list += [value]
city_set = set(city_node_list)
inf_set = gtd_event_city_by_year[1970]
sus_set = city_set - inf_set
inf_list = list(inf_set)
sus_list = list(sus_set)

In [7]:
for cur_year in range(1971, 2016):
    print cur_year
    if cur_year not in gtd_event_city_by_year:
        continue

    year = cur_year - 1971

    new_inf_set = gtd_event_city_by_year[cur_year]
    new_sus_set = city_set - new_inf_set
    new_inf_list = list(new_inf_set)
    new_sus_list = list(new_sus_set)

    for citynode in sus_list:
        try:
            city_num_terror_connected[year][citynode] = 0
            for nbr in city_network.GetNI(citynode).GetOutEdges():
                if nbr in inf_set:
                    city_num_terror_connected[year][citynode] += 1

            city_connected[year][citynode] = city_network.GetNI(citynode).GetDeg()
            city_num_non_terror_connected[year][citynode] = city_connected[year][citynode] - city_num_terror_connected[year][citynode]

            city_terror_occur[year][citynode] = 0

            if citynode in new_inf_set:
                city_terror_occur[year][citynode] = 1
            else:
                city_terror_recover[year][citynode] = -1

            city_betweenness_centrality[year][citynode] = Nodes[citynode]
            city_degree_centrality[year][citynode] = snap.GetDegreeCentr(city_network, citynode)
            city_closeness_centrality[year][citynode] = snap.GetClosenessCentr(city_network, citynode)
            city_farness_centrality[year][citynode] = snap.GetFarnessCentr(city_network, citynode)
        except:
            pass

    for citynode in inf_list:
        try:
            city_num_terror_connected[year][citynode] = 0
            for nbr in city_network.GetNI(citynode).GetOutEdges():
                if nbr in inf_set:
                    city_num_terror_connected[year][citynode] += 1

            city_connected[year][citynode] = city_network.GetNI(citynode).GetDeg()
            city_num_non_terror_connected[year][citynode] = city_connected[year][citynode] - city_num_terror_connected[year][citynode]

            city_terror_recover[year][citynode] = 0
            if citynode in new_sus_set:
                city_terror_recover[year][citynode] = 1
            else:
                city_terror_occur[year][citynode] = -1

            city_betweenness_centrality[year][citynode] = Nodes[citynode]
            city_degree_centrality[year][citynode] = snap.GetDegreeCentr(city_network, citynode)
            city_closeness_centrality[year][citynode] = snap.GetClosenessCentr(city_network, citynode)
            city_farness_centrality[year][citynode] = snap.GetFarnessCentr(city_network, citynode)
        except:
            pass
        
        inf_set = new_inf_set
        sus_set = new_sus_set
        inf_list = new_inf_list
        sus_list = new_sus_list

1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015


In [8]:
X_inf = []
y_inf = []

X_rec = []
y_rec = []

In [9]:
for cur_year in range(1971, 2016):
    print cur_year
    year = cur_year - 1971
    for key, value in city_terror_occur[year].iteritems():
        sample = []
        if value != -1:
            y_inf += [city_terror_occur[year][key]]
            sample += [city_connected[year][key]]
            sample += [city_num_terror_connected[year][key]]
            sample += [city_num_non_terror_connected[year][key]]
            sample += [city_betweenness_centrality[year][key]]
            sample += [city_degree_centrality[year][key]]
            sample += [city_closeness_centrality[year][key]]
            sample += [city_farness_centrality[year][key]]
            X_inf += [sample]

    for key, value in city_terror_recover[year].iteritems():
        sample = []
        if value != -1:
            y_rec += [city_terror_recover[year][key]]
            sample += [city_connected[year][key]]
            sample += [city_num_terror_connected[year][key]]
            sample += [city_num_non_terror_connected[year][key]]
            sample += [city_betweenness_centrality[year][key]]
            sample += [city_degree_centrality[year][key]]
            sample += [city_closeness_centrality[year][key]]
            sample += [city_farness_centrality[year][key]]
            X_rec += [sample]

1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015


In [10]:
clf_inf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [11]:
clf_inf.fit(X_inf, y_inf)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [12]:
clf_rec = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [13]:
clf_rec.fit(X_rec, y_rec)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [14]:
final_inf_list = {}
final_inf_prob = {}

In [15]:
for citynode in sus_list:
    try:
        terror_connected = 0
        for nbr in city_network.GetNI(citynode).GetOutEdges():
            if nbr in inf_set:
                terror_connected += 1

        connected = city_network.GetNI(citynode).GetDeg()
        non_terror_connected = connected - terror_connected

        betweenness = Nodes[citynode]
        degree = snap.GetDegreeCentr(city_network, citynode)
        closeness = snap.GetClosenessCentr(city_network, citynode)
        farness = snap.GetFarnessCentr(city_network, citynode)
        x = [connected, terror_connected, non_terror_connected, betweenness, degree, closeness, farness]

        final_inf_list[citynode] = clf_inf.predict([x])[0]
        final_inf_prob[citynode] = clf_inf.predict_proba([x])[0]
    except:
        pass

In [16]:
for citynode in inf_list:
    try:
        final_inf_list[citynode] = 1

        terror_connected = 0
        for nbr in city_network.GetNI(citynode).GetOutEdges():
            if nbr in inf_set:
                terror_connected += 1

        connected = city_network.GetNI(citynode).GetDeg()
        non_terror_connected = connected - terror_connected

        betweenness = Nodes[citynode]
        degree = snap.GetDegreeCentr(city_network, citynode)
        closeness = snap.GetClosenessCentr(city_network, citynode)
        farness = snap.GetFarnessCentr(city_network, citynode)
        x = [connected, terror_connected, non_terror_connected, betweenness, degree, closeness, farness]

        final_inf_list[citynode] = 1 - clf_rec.predict([x])[0]
        final_inf_prob[citynode] = 1 - clf_rec.predict_proba([x])[0]
    except:
        pass

In [17]:
final_pred_cities = []

In [18]:
for key, value in final_inf_list.iteritems():
	if value == 1:
		try:
			final_pred_cities += [city_list[key]]
		except:
			pass

In [19]:
final_cities = []
for citynode in gtd_event_city_by_year[2016]:
	try:
		final_cities += [city_list[citynode]]
	except:
		pass

In [20]:
print final_pred_cities

['Montreal', 'Toronto', 'Mayo', 'Peterborough', 'Sydney', 'London', 'Annaba', 'Tebessa', 'Ouagadougou', 'San Pedro', 'Abuja', 'Calabar', 'Jos', 'Kaduna', 'Kano', 'Maiduguri', 'Lagos', 'Yola', 'Zaria', 'Diffa', 'Tunis', 'Remada', 'Altenburg', 'Jena', 'Magdeburg', 'Berlin', 'Dresden', 'Munster', 'Hamburg', 'Cologne', 'Leipzig', 'Bielefeld', 'Augsburg', 'Flensburg', 'Oulu', 'Tampere', 'Belfast', 'Londonderry', 'Bristol', 'Glasgow', 'Cork', 'Dublin', 'Copenhagen', 'Bodo', 'Uppsala', 'Trollhattan', 'Stockholm', 'Mora', 'Ljungby', 'Oskarshamn', 'Vilhelmina', 'Orebro', 'Eskilstuna', 'Laage', 'Bethlehem', 'Cape Town', 'Bangui', 'Maroua', 'Maputo', 'Nampula', 'Maseru', 'Goma', 'Bamako', 'Gao', 'Rabat', 'Casablanca', 'Addis Ababa', 'Bujumbura', 'Alexandria', 'Cairo', 'Luxor', 'Port Said', 'Aswan', 'Lamu', 'Mombasa', 'Nairobi', 'Wajir', 'Ghat', 'Benghazi', 'Tripoli', 'Kadugli', 'Juba', 'Malakal', 'Khartoum', 'Tanga', 'Zanzibar', 'Tirana', 'Sofia', 'Paphos', 'Madrid', 'Ocana', 'Santiago', 'Poitier

In [21]:
print final_cities

['Santa Maria', 'Taipei', 'Sanliurfa', 'Princeton', 'Kampala', 'Hamilton', 'Montreal', 'Caceres', 'Toronto', 'Beni', 'Mayo', 'Paysandu', 'Hakkari', 'Samarinda', 'Manila', 'Pagadian', 'Faizabad', 'Lipa', 'Tacloban', 'Bannu', 'Vancouver', 'Dalbandin', 'Rosario', 'Santa Fe', 'Khuzdar', 'Parachinar', 'Turbat', 'Sulaymaniyah', 'London', 'Calgary', 'Posadas', 'General Santos City', 'Jolo', 'Beni Suef', 'Masbate', 'Constantine', 'Yopal', 'Ouagadougou', 'Maracay', 'San Pedro', 'Abuja', 'Gusau', 'Kaduna', 'Maiduguri', 'Makurdi', 'Lagos', 'Diffa', 'Ivano-Frankivsk', 'Rivne', 'Qayyarah', 'Tikrit', 'Taji', 'Brussels', 'Charleroi', 'Grozny', 'Bautzen', 'Vologda', 'Khanabad', 'Magdeburg', 'Berlin', 'Dresden', 'Hamburg', 'Munich', 'Leipzig', 'Shillong', 'Guwahati', 'Dimapur', 'Bharatpur', 'Temuco', 'Nepalgunj', 'Essen', 'Parma', 'Georgetown', 'Bogota', 'Leer', 'Cucuta', 'Cartagena', 'Florencia', 'Cooktown', 'Tame', 'Arauca', 'Santa Ana', 'Belfast', 'Londonderry', 'Malakal', 'San Ramon', 'Plymouth', '

In [22]:
right_count = 0
for city in final_pred_cities:
    print city
    if city in final_cities:
        right_count += 1

print right_count * 1.0/len(final_pred_cities)

Montreal
Toronto
Mayo
Peterborough
Sydney
London
Annaba
Tebessa
Ouagadougou
San Pedro
Abuja
Calabar
Jos
Kaduna
Kano
Maiduguri
Lagos
Yola
Zaria
Diffa
Tunis
Remada
Altenburg
Jena
Magdeburg
Berlin
Dresden
Munster
Hamburg
Cologne
Leipzig
Bielefeld
Augsburg
Flensburg
Oulu
Tampere
Belfast
Londonderry
Bristol
Glasgow
Cork
Dublin
Copenhagen
Bodo
Uppsala
Trollhattan
Stockholm
Mora
Ljungby
Oskarshamn
Vilhelmina
Orebro
Eskilstuna
Laage
Bethlehem
Cape Town
Bangui
Maroua
Maputo
Nampula
Maseru
Goma
Bamako
Gao
Rabat
Casablanca
Addis Ababa
Bujumbura
Alexandria
Cairo
Luxor
Port Said
Aswan
Lamu
Mombasa
Nairobi
Wajir
Ghat
Benghazi
Tripoli
Kadugli
Juba
Malakal
Khartoum
Tanga
Zanzibar
Tirana
Sofia
Paphos
Madrid
Ocana
Santiago
Poitiers
Auch
Ajaccio
Macon
Marseille
Nice
Orange
Paris
Le Mans
Heraklion
Thessaloniki
Budapest
Milan
Bologna
Brescia
Rome
Prague
Ankara
Adana
Gaziantep
Iskenderun
Malatya
Istanbul
Izmir
Erzurum
Trabzon
Van
Batman
Siirt
Skopje
San Jose
Acapulco
Matamoros
Leon
Buenos Aires
Herat
Jalala

In [23]:
import plotly.plotly as py
from plotly.graph_objs import *

In [24]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [25]:
print final_inf_prob

{0: array([ 0.69493211,  0.30506789]), 1: array([ 0.80095247,  0.19904753]), 2: array([ 0.77994796,  0.22005204]), 3: array([ 0.82198479,  0.17801521]), 4: array([ 1.,  0.]), 5: array([ 0.70106895,  0.29893105]), 6: array([ 1.,  0.]), 7: array([ 1.,  0.]), 8: array([ 1.,  0.]), 9: array([ 0.50595839,  0.49404161]), 10: array([ 0.50595839,  0.49404161]), 11: array([ 0.50595839,  0.49404161]), 14: array([ 0.50595839,  0.49404161]), 15: array([ 1.,  0.]), 17: array([ 1.,  0.]), 20: array([ 0.62250752,  0.37749248]), 21: array([ 1.,  0.]), 22: array([ 1.,  0.]), 23: array([  1.00000000e+000,   1.08403706e-128]), 26: array([ 0.66729169,  0.33270831]), 27: array([ 0.66730273,  0.33269727]), 28: array([ 0.70179866,  0.29820134]), 29: array([  1.00000000e+000,   6.12762838e-130]), 30: array([ 0.514038,  0.485962]), 31: array([ 0.69159734,  0.30840266]), 32: array([ 0.62524457,  0.37475543]), 33: array([ 0.62524457,  0.37475543]), 35: array([ 0.54103794,  0.45896206]), 36: array([ 1.,  0.]), 39

In [26]:
print final_inf_list

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 14: 0, 15: 0, 17: 0, 20: 0, 21: 0, 22: 0, 23: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 35: 0, 36: 0, 39: 0, 40: 0, 44: 0, 47: 0, 48: 0, 49: 0, 53: 0, 54: 0, 55: 0, 57: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 1, 71: 0, 73: 0, 75: 0, 76: 0, 77: 0, 78: 0, 80: 1, 83: 0, 85: 0, 86: 1, 88: 0, 89: 0, 91: 0, 96: 0, 97: 0, 102: 0, 103: 0, 104: 1, 105: 0, 106: 0, 108: 0, 109: 0, 110: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 121: 0, 122: 1, 123: 0, 124: 0, 128: 0, 129: 0, 131: 0, 132: 0, 135: 0, 136: 0, 137: 0, 138: 0, 139: 0, 140: 0, 141: 0, 142: 0, 143: 0, 146: 0, 147: 0, 148: 0, 149: 0, 150: 0, 154: 0, 155: 0, 157: 0, 158: 0, 159: 0, 160: 0, 161: 0, 162: 0, 164: 0, 165: 0, 166: 1, 167: 0, 168: 0, 169: 0, 170: 0, 171: 0, 172: 0, 173: 0, 174: 0, 175: 0, 176: 0, 179: 0, 180: 0, 181: 0, 182: 0, 184: 0, 186: 0, 188: 0, 189: 0, 190: 0, 192: 0, 195: 0, 19

In [ ]:
final_pred_coords = []
for key, value in final_inf_list.iteritems():
    print key
    if value == 1:
        try:
            location = geolocator.geocode(city_list[key])
            final_pred_coords += [(final_inf_prob[key][0], location.latitude, location.longitude)]
        except:
            pass

In [ ]:
print city_list